In [ ]:
# Por si alguien corre en python2
from __future__ import division

# Preparamos todo para correr
import numpy as np
from math import sqrt
from matplotlib import pylab as plt

# *Probabilidades condicionales y el teorema de Bayes*

### Un ejemplo

Una enfermedad genética rara afecta al 0.1% de la población de un país. Para diagnosticarla, existe un análisis clínico que detecta la enfermedad en el 99% de los casos en los que el paciente la padece, y tiene un 1% de falsos positivos.

Un paciente recibe un resultado positivo del análisis clínico. 

<ul>
    <li>¿Qué probabilidad hay de que la persona padezca la enfermedad?</li>
    <li>¿Cuál sería el próximo paso natural?</li>
</ul>

*****

En primer lugar, pasemos a probabilidades los datos que tenemos:

<ol>
    <li> La probabilidad de tener la enfermedad es 1%; $P(E) = 0.01$</li>
    <li> Si un paciente tiene la enfermedad, el análisis da positivo en el 99% de los casos; $P(D | E) = 0.99$</li>
    <li> El porcentaje de falsos positivos del diagnóstico es 1%; $P(D | \bar{E}) = 0.01$</li>
    </ol>
    
Lo que nos preguntamos es cuál es la probabilidad de que el paciente tenga la enfermedad, dado que recibió un resultado positivo. Buscamos entonces $P(E | D)$. <b>¿Cómo se escribe esto en términos de las cantidades que conocemos?</b>

La respuesta se obtiene a partir del teorema de Bayes:

$$
P(E | D) = \frac{P(D | E)\,P(E)}{P(D)}
$$

Muy bien, pero nos falta conocer $P(D)$; para eso, usamos la regla de la suma:

$$
P(D) = P(D | E)\,P(E) + P(D | \bar{E}) P(\bar{E})
$$

Poniendo todo junto:

$$
P(E | D) = \frac{P(D | E)\,P(E)}{P(D | E)\,P(E) + P(D | \bar{E}) P(\bar{E})}
$$

Y ahora sí podemos calcular.

In [ ]:
#Escribo las diferentes probabilidades
pE = 0.001
pDE = 0.99
pDnE = 0.01

pED = pDE * pE / (pDE * pE + pDnE * (1 - pE))

print('La probabilidad de que el paciente tenga la enfermedad es {:.4f}'.format(pED))

El paciente puede sentirse inquieto por el resultado, pero la probabilidad sigue siendo relativamente baja. El paso obvio es repetir el test. Calculemos cuál es la probabilidad de tener la enfermedad si dos tests son positivos.

De nuevo, tenemos que escribir el teorema de Bayes

$$
p(E|DD) = \frac{P(DD | E) P(E)}{P(DD)}\;\;.
$$

Acá, $P(E)$ sigue siendo lo mismo que arriba, la probabilidad de tener la enfermedad. Además, podemos escribir, usando la regla del producto

$$P(DD | E) = P(D |ED)P(D | E) = P(D|E)P(D|E)=p(D|E)^2\;\;,$$

donde el último paso viene del hecho de que el resultado de un test es independiente del resultado anterior. Además, tenemos que calcular $P(DD)$. Al igual que antes, usamos la ley de la probabilidad total (marginalización).

$$
P(DD) = P(DD|E)P(E) + P(DD|\bar{E})P(\bar{E}) = P(D|E)^2 P(E) + P(D|\bar{E})^2 P(\bar{E})\;\;,
$$
donde la última línea vale por la independencia de los tests.

In [ ]:
pDDE = pDE**2
pDD = pDE**2 * pE + pDnE**2 * (1-pE)

print('La probabilidad de dos tests positivos, si el paciente está enfermo, es P(DD|E) = {:.4f}'.format(pDDE))
print('La probabilidad de dos tests positivos, independientemente del estado del paciente, es P(DD) = {:.4f}'.format(pDD))

A primera vista, el resultado para $P(DD|E)$ puede parecer intuitivo, porque es menor que $P(D|E)$, pero recuerden que lo que _realmente_ estamos preguntando es $P(E|DD)$.

In [ ]:
pEDD = pDDE*pE / pDD

print('Ante dos tests positivos, la probabilidad de tener la enfermedad es {:4f}.'.format(pEDD))

#### Pregunta
<ul>
    <li>¿Cómo cambia la probabilidad si el segundo test fuera negativo?</li>
</ul>

***
## Caso: Apruebo si falto a clase?

Tomaremos un set de Datos de _Student Alcohol Consumption_ de la UCI alojada por Kaggle. Pueden descargar el dataset de la clase de matemáticas aquí https://www.kaggle.com/uciml/student-alcohol-consumption#student-mat.csv y subirlo en google Colab, desde la pestaña izquierda. En el repo, ya esta alojado en el directorio _datasets_.

Para los que usan Google Colab, ejecuten esta celda una unica vez para descargar los datos. Si clonaron el repositorio usando git, no hace falta.

In [ ]:
#Si estas en Google Colab, ejecuta esta celda una unica vez:
!mkdir datasets
!mkdir datasets/student-alcohol-consumption
!wget https://raw.githubusercontent.com/exord/UNSAM_IA/master/datasets/student-alcohol-consumption/student-mat.csv
!mv student-mat.csv datasets/student-alcohol-consumption/student-mat.csv

Ahora vamos a importar las librerias que usaremos:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Primero que nada, carguemos el dataset usando pandas:

In [ ]:
df = pd.read_csv('datasets/student-alcohol-consumption/student-mat.csv')
print("Cantidad de registros: {}".format(len(df)))
df.head()

Los datos son de una encuesta realizada a 395 alumnos de una clase de matemáticas, y tenemos muchos campos. Nos concentraremos en la siguiente pregunta: **¿Cual es la probabilidad de aprobar con una nota de 80% o más si falté 10 o más clases?**

Para hacerlo, crearemos unas columnas con 0 o 1 dependiendo de si cumplen las condiciones o no, así facilitaremos la tarea de contabilizar. La nota final esta en la columna _G3_, en una escala de 0 a 20. Las ausencias se contabilizan en _absences_.

In [ ]:
df['aprueba'] = np.where(df['G3']*5 >= 80, 1, 0)
df['falto_mucho'] = np.where(df['absences'] >= 10, 1, 0)

Para contar las columnas, nos será útil crear una columna de _1_ constante, así contar es simplemente sumar dicho campo.

In [ ]:
df['cuenta'] = 1

Ahora nos deshacemos de todas las otras columnas:

In [ ]:
df = df[['aprueba','falto_mucho','cuenta']]
df.head()

Lo que sigue es transformar nuestra tabla. Queremos una tabla 2x2, la cuyos ejes sean el valor de "aprueba" y "falto mucho", y cuyos contenidos sea la cantidad de alumnos en cada categoría. Eso lo podemos conseguir utilizando el método `pd.pivot_table` usando la suma como función de agregación de la columna _cuenta_:

La funcion `pivot_table` toma un `DataFrame` y crea a partir de el otro que tenga como índices los valores de la columna indicada en `index=`, como columnas los valores de las columnas indicadas en `columns=`, y como valores a los de las columnas indicadas en `values=`. Si hay muchas coincidencias de valores para una misma celda, forma un vector. Para obtener un número en cada celda, seleccionamos una función de agregación con `aggfunc=` que se aplicará sobre los vectores. En caso de que hayan celdas vacías, se rellenarán usando el valor especificado en `fill_value=`.

En nuestro caso, como usamos de valores la columna _cuenta_ que tiene solo números 1, la función de agregación para contar casos puede ser bien la de la suma `np.sum`. (Prueben también con `len`, la longitud del vector, que debería funcionar de igual modo).

In [ ]:
pd.pivot_table(
    df,
    values = 'cuenta',
    index = ['aprueba'],
    columns = ['falto_mucho'],
    aggfunc = np.sum,
    fill_value = 0)

Podemos ver entonces que, la cantidad de alumnos que falto poco y desaprobó es 277, que faltó mucho y desaprobó es 78, que faltó poco y aprobó es 35 y que faltó mucho y aprobó es 5, siendo la cantidad total de alumnos 395. Definimos las probabilidades entonces:

$$ P(\text{aprueba}) = \frac{35 + 5}{395} \approx  0.10126582278481013$$ 
$$ P(\text{falto_mucho}) = \frac{78 + 5}{395} \approx 0.21012658227848102$$
$$ P(\text{aprueba} \cap \text{falto_mucho}) = \frac{5}{395} \approx  0.012658227848101266$$

La probabilidad condicional de que hayan aprobado, dado que faltaron mucho, es por definición:
$$  P(\text{aprueba }\, |\, \text{ falto_mucho}) = \frac{P(\text{aprueba} \cap \text{falto_mucho})}{P(\text{falto_mucho})} $$

Es decir:

In [ ]:
print("Probabilidad de sacar un 80% o mas dado que falte 10 o mas clases: {:.2f}%".format(5/(78+5)*100))